In [1]:
import torch
import torch.nn as nn

# Define the same LSTM model architecture
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]  # take last time step
        out = self.fc(out)
        return out

input_dim = 11  # number of indicators
hidden_dim = 64
num_layers = 2
output_dim = 3  # 0=down, 1=no, 2=up

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim)
model.load_state_dict(torch.load("crypto_reversal_model.pth"))
model.eval()


CryptoLSTM(
  (lstm): LSTM(11, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=3, bias=True)
)

In [6]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler

# Example feature values
example_features = {
    "RSI": 47.186692,
    "MACD": 74.503073,
    "Signal": 62.507109,
    "Hist": 11.995964,
    "DEMA": 117669.694874,
    "DEMA_Signal": -1,
    "SMA": 117690.536667,
    "SMA_Signal": 1,
    "TSI": 18.261580,
    "%K": 72.867065	,
    "%D": 70.357798
}

FEATURES = ["RSI", "MACD", "Signal", "Hist", "DEMA", "DEMA_Signal", 
            "SMA", "SMA_Signal", "TSI", "%K", "%D"]

# Convert to array in correct order
X_input = np.array([example_features[f] for f in FEATURES], dtype=np.float32)

# Scale features (if using scaler from training, load it)
scaler = StandardScaler()
X_input_scaled = scaler.fit_transform(X_input.reshape(1, -1))  # shape: (1, num_features)

# Add batch and sequence dimensions for LSTM
X_input_scaled = torch.tensor(X_input_scaled, dtype=torch.float32).unsqueeze(0)  # shape: (batch=1, seq_len=1, features=11)


In [8]:
df.head()

NameError: name 'df' is not defined

In [7]:
with torch.no_grad():
    output = model(X_input_scaled)
    predicted_label = torch.argmax(output, dim=1).item()

label_map = {0: "Downtrend Reversal", 1: "No Reversal", 2: "Uptrend Reversal"}
print("Predicted label:", label_map[predicted_label])


Predicted label: No Reversal
